<a href="https://colab.research.google.com/github/Pranav-2509/NIGHT_VISION_SELF/blob/main/Vision_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-------------------------
#Imports
-------------------------

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

--------------------------
#Device Agnostic Code
-------------------------

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
print (device)

cuda


------------------------
#Loading the Data
-----------------------

In [ ]:
train_data = pd.read_csv('train_dataset.csv')

In [ ]:
print (train_data)

        id                                             pixels  emotion
0        1  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...        0
1        2  151 150 147 155 148 133 111 140 170 174 182 15...        0
2        3  231 212 156 164 174 138 161 173 182 200 106 38...        2
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...        4
4        5  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...        6
...    ...                                                ...      ...
4995  4996  22 24 23 23 25 24 23 20 18 13 6 2 0 1 7 22 32 ...        3
4996  4997  73 85 87 87 74 118 120 132 134 127 133 118 105...        3
4997  4998  253 253 254 254 254 254 250 219 166 141 109 70...        6
4998  4999  78 84 77 95 90 85 72 75 79 84 86 82 88 102 110...        6
4999  5000  98 100 102 104 107 109 111 119 126 130 53 5 12...        3

[5000 rows x 3 columns]


In [ ]:
print (train_data.dtypes)

id          int64
pixels     object
emotion     int64
dtype: object


In [ ]:
values_list = train_data.iloc [0, 1]
values_list = values_list.split()

In [ ]:
values_list2 = [float (x) for x in values_list]
values_list2 = np.array (values_list2)
print (values_list2.reshape ((1, 48, 48)))

[[[ 70.  80.  82. ...  52.  43.  41.]
  [ 65.  61.  58. ...  56.  52.  44.]
  [ 50.  43.  54. ...  49.  56.  47.]
  ...
  [ 91.  65.  42. ...  72.  56.  43.]
  [ 77.  82.  79. ... 105.  70.  46.]
  [ 77.  72.  84. ... 106. 109.  82.]]]


# Converting the data into a usable format

In [ ]:
train_data_tuples = []

In [ ]:
for i in range (5000):
  pixel_vals = train_data.iloc [i, 1]
  pixel_vals = pixel_vals.split()
  pixel_vals = [float (x) for x in pixel_vals]
  pixel_vals = [x/255.0 for x in pixel_vals]
  pixel_vals = [(x - 0.5) / 0.5 for x in pixel_vals]
  pixel_vals = np.array (pixel_vals)
  pixel_vals = pixel_vals.reshape ((1, 48, 48))
  pixel_vals = torch.from_numpy (pixel_vals)
  label = train_data.iloc [i, 2]
  pair = (pixel_vals, label)
  train_data_tuples.append (pair)

In [ ]:
train_data_tuples[0]

(tensor([[[-0.4510, -0.3725, -0.3569,  ..., -0.5922, -0.6627, -0.6784],
          [-0.4902, -0.5216, -0.5451,  ..., -0.5608, -0.5922, -0.6549],
          [-0.6078, -0.6627, -0.5765,  ..., -0.6157, -0.5608, -0.6314],
          ...,
          [-0.2863, -0.4902, -0.6706,  ..., -0.4353, -0.5608, -0.6627],
          [-0.3961, -0.3569, -0.3804,  ..., -0.1765, -0.4510, -0.6392],
          [-0.3961, -0.4353, -0.3412,  ..., -0.1686, -0.1451, -0.3569]]],
        dtype=torch.float64),
 0)

In [ ]:
training_data = train_data_tuples[:4000]
validation_data = train_data_tuples[4000:5000]

In [ ]:
# import torch
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader

# # Define a transform to normalize the data
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
# ])


# features = torch.tensor(train_data.iloc[:, :-1].values, dtype=torch.float32)
# # Assuming the last column is the target
# targets = torch.tensor(train_data.iloc[:, -1].values, dtype=torch.float32)

# train_data = TensorDataset(features, targets)
# train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# # # Load the MNIST dataset
# # train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
# # test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# # # Define batch size
# # batch_size = 64

# # print (type(train_dataset))
# # # Create DataLoaders for training and test sets
# # trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# # testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# # Verify the shape of the batches
# for images, labels in train_loader:
#     print(images.shape, labels.shape)
#     break


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
#Dataloader creation

train_loader = DataLoader (training_data, batch_size = 64, shuffle = True)
test_loader = DataLoader (validation_data, batch_size = 64, shuffle = True)

In [ ]:
for batch, (data, label) in enumerate (train_loader):
  print (data.shape)
  print (label.shape)
  break

torch.Size([64, 1, 48, 48])
torch.Size([64])


In [ ]:
num_classes = 7

#Loading the CIFAR - 100 dataset

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5 ,0.5, 0.5))
])
batch_size = 128

# Download and load the CIFAR-100 training set
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# Download and load the CIFAR-100 test set
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Classes in CIFAR-100
classes = trainset.classes
num_classes = len (classes)

# Example to iterate through the training set
dataiter = iter(train_loader)
images, labels = next(dataiter)

print('Loaded CIFAR-10 dataset')
print('Batch of images shape:', images.shape)
print('Batch of labels shape:', labels.shape)


Files already downloaded and verified
Files already downloaded and verified
Loaded CIFAR-10 dataset
Batch of images shape: torch.Size([128, 3, 32, 32])
Batch of labels shape: torch.Size([128])


-------------------------
#Splitting image into tokens
-------------------------

# Patch Tokenizer

In [ ]:
patch_size = 8
image_dim = images.shape[-1]
token_length = 50

In [ ]:
class Patch_Tokenization (nn.Module):
  def __init__ (self, img_size, patch_size : int = 50, token_length : int = 768):

    super().__init__()

    self.img_size = img_size
    c, h, w = self.img_size
    self.patch_size = patch_size
    self.token_length = token_length

    self.no_tokens = (h*w)/(patch_size**2)

    #Layers
    self.split = nn.Unfold (kernel_size = self.patch_size,stride = self.patch_size, padding = 0)
    self.linear = nn.Linear (in_features = c * (self.patch_size)**2, out_features = self.token_length)


  def forward (self, img):
    img = self.split (img).transpose(1, 2)
    img =  img.to(dtype=torch.float32)
    img = self.linear (img)
    return img

In [ ]:
tokeniser = Patch_Tokenization (images[0].shape, patch_size = patch_size, token_length = token_length)
tokeniser = tokeniser.to(device)

In [ ]:
num_tokens = int ((image_dim/patch_size )**2)       #If its not int, then it will be float

--------------------
#Position Encoding
-------------------

In [ ]:
def position_embedding (num_tokens : int, token_length : int):
  positions = torch.zeros ((num_tokens, token_length))
  for i in range (num_tokens):
    for j in range (token_length):
      angle = i/ (np.power(1000, (2* (j//2))/token_length))
      if (j%2==0):
        positions[i][j] = np.sin (angle)
      else:
        positions[i][j] = np.cos (angle)

  return positions

In [ ]:
position_encoding = position_embedding (num_tokens+1, token_length)

In [ ]:
position_encoding = position_encoding.unsqueeze (dim = 0)
print (position_encoding.shape)

torch.Size([1, 17, 50])


In [ ]:
print (position_encoding)
position_encoding = position_encoding.to(device)
print (position_encoding.device)

tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,
           1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
           0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,
           1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
           0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,
           1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,
           0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,
           1.0000],
         [ 0.8415,  0.5403,  0.6879,  0.7258,  0.5442,  0.8390,  0.4228,
           0.9062,  0.3251,  0.9457,  0.2486,  0.9686,  0.1894,  0.9819,
           0.1440,  0.9896,  0.1094,  0.9940,  0.0831,  0.9965,  0.0631,
           0.9980,  0.0478,  0.9989,  0.0363,  0.9993,  0.0275,  0.9996,
           0.0209,  0.9998,  0.0158,  0.9999,  0.0120,  0.9999,  0.0091,
           1.0000,  0.0069,  1.0000,  0.0052,  1.0000,  0.0040,  1.0000,
           0.0030,  1.0000,  0.

------------------
#Attention Block
------------------

In [ ]:
class Attention(nn.Module):
    def __init__(self,
                dim: int,
                chan: int,
                num_heads: int=1,
                qkv_bias: bool=False,
                qk_scale = None):

        """ Attention Module

            Args:
                dim (int): input size of a single token
                chan (int): resulting size of a single token after concatenating the heads
                num_heads(int): number of attention heads in MSA
                qkv_bias (bool): determines if the qkv layer learns an addative bias
                qk_scale (NoneFloat): value to scale the queries and keys by;
                                    if None, queries and keys are scaled by ``head_dim ** -0.5``
        """

        super().__init__()

        ## Define Constants
        self.num_heads = num_heads
        self.chan = chan
        self.head_dim = self.chan // self.num_heads
        self.scale = qk_scale or self.head_dim ** -0.5
        assert self.chan % self.num_heads == 0, '"Chan" must be evenly divisible by "num_heads".'

        ## Define Layers
        self.layer_norm = nn.LayerNorm (dim)
        self.q = nn.Linear(dim, chan , bias=qkv_bias)
        self.k = nn.Linear (dim, chan, bias = qkv_bias)
        self.v = nn.Linear (dim, chan, bias = qkv_bias)

        #### Each token gets projected from starting length (dim) to channel length (chan) 3 times (for each Q, K, V)
        self.proj = nn.Linear(chan, chan)



    #     self._initialize_weights()

    # def _initialize_weights(self):
    #     for m in self.modules():
    #         if isinstance(m, nn.Linear):
    #             torch.nn.init.kaiming_uniform_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
    #             if m.bias is not None:
    #                 nn.init.zeros_(m.bias)

    def forward(self, x):
        B, N, C = x.shape
        ## Dimensions: (batch, num_tokens, token_len)


        x = self.layer_norm (x)

        ## Calcuate QKVs

        # qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        #### Dimensions: (3, batch, heads, num_tokens, chan/num_heads = head_dim)
        q_matrix, k_matrix, v_matrix = self.q (x), self.k (x), self.v (x)
        q_matrix = q_matrix.reshape (B, self.num_heads, N, self.head_dim)
        k_matrix = k_matrix.reshape (B, self.num_heads, N, self.head_dim)
        v_matrix = v_matrix.reshape (B, self.num_heads, N, self.head_dim)


        ## Calculate Attention
        attn = (q_matrix * self.scale) @ k_matrix.transpose(-2, -1)
        attn = attn.softmax(dim=-1)
        #### Dimensions: (batch, heads, num_tokens, num_tokens)

        ## Attention Layer
        x = (attn @ v_matrix).reshape(B, N, self.chan)
        #### Dimensions: (batch, heads, num_tokens, chan)

        ## Projection Layers
        x = self.proj(x)
        ## Skip Connection Layer
        v_matrix = v_matrix.reshape(B, N, self.chan)
        x = v_matrix + x
        #### Because the original x has different size with current x, use v to do skip connection
        return x

In [ ]:
# sample_input = torch.rand ((5, 17, 50))
# e = Attention (50, 100, 2)
# output = e (sample_input)
# print (output.shape)

----------------------------------
#Neural Net at the end of the Encoder
-----------------------------------

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self,
       in_chan: int,
       hidden_chan = None,
       out_chan = None,
       act_layer = nn.GELU()):

        super().__init__()

        ## Define Number of Channels
        hidden_chan = hidden_chan
        out_chan = out_chan

        ## Define Layers
        self.fc1 = nn.Linear(in_chan, hidden_chan)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_chan, out_chan)


    def forward(self, x):

        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)

        return x

-----------------------------------
#Probabilities extraction
------------------------------------


In [ ]:
class probabilities (nn.Module ):
  def __init__ (self,in_size : int, num_classes : int):

    super().__init__()

    self.layer1 = nn.Linear (in_size, 70)
    self.activation = nn.ReLU()
    self.layer2 = nn.Linear (70, num_classes)

  #   self._initialize_weights()

  # def _initialize_weights(self):
  #       for m in self.modules():
  #           if isinstance(m, nn.Linear):
  #               torch.nn.init.kaiming_uniform_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
  #               if m.bias is not None:
  #                   nn.init.zeros_(m.bias)


  def forward (self, x):
    x =  (self.layer1(x))
    x = self.activation (x)
    x = (self.layer2(x))
    return x

--------------------------------
#Encoding Block
--------------------------------

In [ ]:
class Encoding(nn.Module):

    def __init__(self,
       dim: int,
       num_heads: int=1,
       hidden_chan_mul = 4.,
       qkv_bias = False,
       qk_scale = None,
       act_layer=nn.ReLU,
       norm_layer=nn.LayerNorm):

        """ Encoding Block

            Args:
                dim (int): size of a single token
                num_heads(int): number of attention heads in MSA
                hidden_chan_mul (float): multiplier to determine the number of hidden channels (features) in the NeuralNet component
                qkv_bias (bool): determines if the qkv layer learns an addative bias
                qk_scale (NoneFloat): value to scale the queries and keys by;
                                    if None, queries and keys are scaled by ``head_dim ** -0.5``
                act_layer(nn.modules.activation): torch neural network layer class to use as activation
                norm_layer(nn.modules.normalization): torch neural network layer class to use as normalization
        """

        super().__init__()

        ## Define Layers
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim=dim,
                            chan=dim,
                            num_heads=num_heads,
                            qkv_bias=qkv_bias,
                            qk_scale=qk_scale)
        self.dropout = nn.Dropout (p = 0.5)
        self.norm2 = norm_layer(dim)
        self.neuralnet = NeuralNet(in_chan=dim,
                                hidden_chan=int(dim*hidden_chan_mul),
                                out_chan=dim,
                                act_layer=act_layer)

        self.probabs = probabilities (dim, num_classes)

    #     self._initialize_weights()

    # def _initialize_weights(self):
    #     for m in self.modules():
    #         if isinstance(m, nn.Linear):
    #             torch.nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
    #             if m.bias is not None:
    #                 nn.init.zeros_(m.bias)


    def forward(self, x):
        x = x + self.dropout (self.attn((x)))
        # print ('1', x)
        x = x + self.dropout (self.neuralnet(self.norm2(x)))
        # print ('2', x)
        x = x[:, 0]
        x = self.probabs (x)
        # print ('3', x)
        return x

In [ ]:
encoder  = Encoding (dim = token_length, num_heads = 2, hidden_chan_mul = 1, qkv_bias = True)
encoder = encoder.to (device)

----------------------------
#Training Loop
---------------------------

In [ ]:
loss_fn = nn.CrossEntropyLoss ()

In [ ]:
import torch.optim as optim
optimizer = optim.SGD(encoder.parameters(), lr=0.1, momentum = 0.9)

In [ ]:
epochs = 100
num_batches = len (train_loader)

for i in range (epochs):
  print (f'EPOCH : {i+1}')

  tot = 0
  correct = 0
  train_loss = 0
  test_loss = 0
  test_acc = 0
  for batch, (X, y) in enumerate (train_loader):

    # print (batch)

    X = X.to(device)
    y = y.to(device)
    if (batch == num_batches - 1):
      break

    encoder.train()
    # print (X.dtype)
    X = tokeniser (X)
    # print (X)
    zero_tensor = torch.zeros ((batch_size, 1, token_length))
    zero_tensor = zero_tensor.to (device)

    X_appended = torch.concat ((zero_tensor, X), dim = 1)
    # print (X)
    X_positional_encoded = X_appended + position_encoding
    X_post_encoder = encoder (X_positional_encoded)
    X_post_encoder = encoder (X_positional_encoded)

    # break

    optimizer.zero_grad()
    loss = loss_fn (X_post_encoder, y)
    train_loss+=loss
    loss.backward()
    optimizer.step()

  # break

  print (f'Train Loss for batch : {train_loss/len (train_loader)}')
  test_loss = 0
  for batch, (X, y) in enumerate (test_loader):

    tot +=len (X)
    if (batch == len (test_loader) - 1):
      break

    encoder.eval()

    X = X.to(device)
    y = y.to (device)

    X = tokeniser (X)
    zero_tensor = torch.zeros ((batch_size, 1, token_length))
    zero_tensor = zero_tensor.to (device)
    X_appended = torch.concat ((zero_tensor, X), dim = 1)
    X_positional_encoded = X_appended + position_encoding
    X_post_encoder = encoder (X_positional_encoded)
    X_post_encoder = encoder (X_positional_encoded)

    test_loss += loss_fn (X_post_encoder, y)
    X_post_encoder = torch.argmax ((X_post_encoder), dim = 1)

    correct += (X_post_encoder == y).sum().item()
  print (f'test_loss per batch = {test_loss/len (test_loader)}')
  print (f'test accuracy : {correct/tot}')











EPOCH : 1
Train Loss for batch : 1.8826812505722046
test_loss per batch = 1.8067229986190796
test accuracy : 0.3156
EPOCH : 2
Train Loss for batch : 1.8805748224258423
test_loss per batch = 1.767155408859253
test accuracy : 0.3345
EPOCH : 3
Train Loss for batch : 1.8747856616973877
test_loss per batch = 1.7641940116882324
test accuracy : 0.3453
EPOCH : 4
Train Loss for batch : 1.8778167963027954
test_loss per batch = 1.7859960794448853
test accuracy : 0.3255
EPOCH : 5
Train Loss for batch : 1.8644273281097412
test_loss per batch = 1.7845125198364258
test accuracy : 0.3335
EPOCH : 6
Train Loss for batch : 1.8698080778121948
test_loss per batch = 1.7712483406066895
test accuracy : 0.3278
EPOCH : 7
Train Loss for batch : 1.865853190422058
test_loss per batch = 1.806781530380249
test accuracy : 0.3326
EPOCH : 8
Train Loss for batch : 1.8687809705734253
test_loss per batch = 1.82599937915802
test accuracy : 0.3075
EPOCH : 9
Train Loss for batch : 1.8718289136886597
test_loss per batch = 1.7

KeyboardInterrupt: 

------------------
#Experimentation
------------------

In [ ]:
for batch, (X, y) in enumerate (test_loader):
  print (X)
  break

KeyError: 312

In [ ]:
# import torch
# import torch.nn as nn

# patch_size = 50
# unfold = nn.Unfold(kernel_size=patch_size, stride=patch_size, padding=0)

In [ ]:
# x = torch.randn(1, 1, 100, 100)  # Batch size of 1, 1 channel, 100x100 image

In [ ]:
# patches = unfold(x)

In [ ]:
# print (x)

In [ ]:
# print (patches)

In [ ]:
# layer = nn.Linear (in_features = 2500, out_features = 768)

In [ ]:
# input = torch.randn((100, 2500))
# print (input)

In [ ]:
# output = layer(input)
# print (output.shape)

In [ ]:
# batch, sentence_length, embedding_dim = 20, 5, 10
# embedding = torch.randn(batch, sentence_length, embedding_dim)
# layer_norm = nn.LayerNorm(embedding_dim)
# embedding_2 = layer_norm(embedding)

In [ ]:
# embedding_1 = embedding[0][0]
# mean = embedding_1.mean()

In [ ]:
# var = torch.var (embedding_1)

In [ ]:
# embedding_1 = (embedding_1 - mean)/(var)**1/2
# print (embedding_1)
# print (embedding_2[0][0])

experimentation for attention

In [ ]:
# a = torch.rand (3, 4, 12)
# b = a.reshape (3, 3, 2 , 4, 2)

In [ ]:
# print (a)

In [ ]:
# print (b)

checking how attention weights are calculated

In [ ]:
batch_size, num_heads, num_tokens, head_dim = 2, 4, 5, 6

q = torch.randint (1, 10, (batch_size, num_heads, num_tokens, head_dim))
q = q.float()

k = torch.randint (1, 10, (batch_size, num_heads, num_tokens, head_dim))
k = k.float()

In [ ]:
print (q, '\n', k)

In [ ]:
print (k.transpose (-2, -1))

In [ ]:
print (q.shape)

weights = q@ (k.transpose (-2, -1))

print (weights.shape)

In [ ]:
weights = weights.softmax (dim = -1)
print (weights.shape)

In [ ]:
print (q)

In [ ]:
q_modified = q.transpose (1, 2)
print (q_modified)

In [ ]:
q_ultra_modified = q_modified.reshape (batch_size, num_tokens, num_heads*head_dim)
print (q_ultra_modified)